In [1]:
import time

import astar
import utils

import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib qt

In [4]:
ROAD_WIDTH = 5
TRACK_SIZE = 20
INTERSECTION_SIZE = 6
FLOOR_GAP = 10 # distance between floors
STAIR_SIZE = int(FLOOR_GAP/2)
NUM_FLOORS = 5

CURR_ELV_LOC = 0 * FLOOR_GAP
ELV_SPEED = 1
ROBOT_SPEED = 1

voxel_size = [1, 1, 1]

floors_list = []
for i in range(NUM_FLOORS):
    floors_list.append(i*FLOOR_GAP)

In [5]:
# Map building

road_list = []
stair_list = []
corridor_list = []
elevator_list = []
for i in range(TRACK_SIZE+1):
    for j in range(ROAD_WIDTH):
        for z in floors_list:
            point = np.array([i,j,z])
            road_list.append(point)
            corridor_list.append(point)
for i in range(INTERSECTION_SIZE):
    for j in range(ROAD_WIDTH):
        r_x = TRACK_SIZE + 1 + i
        for z in floors_list:
            point = np.array([r_x, j, z])
            road_list.append(point)
            corridor_list.append(point)
        if j <= int(ROAD_WIDTH / 2):
            for n in range(NUM_FLOORS - 1):
                point = np.array([r_x, j + ROAD_WIDTH + STAIR_SIZE - 1, int(FLOOR_GAP/2) + FLOOR_GAP * n])
                road_list.append(point)
                stair_list.append(point)
    for n in range(NUM_FLOORS - 1):
        for k in range(STAIR_SIZE - 1):
            r_y = ROAD_WIDTH + k
            if i >= INTERSECTION_SIZE / 2:
                r_z = int(FLOOR_GAP/2) * k / STAIR_SIZE + 1 + n * FLOOR_GAP
            else:
                r_z = (n+1) * FLOOR_GAP - int(FLOOR_GAP/2) * k / STAIR_SIZE - 1
            point = np.array([r_x, r_y, r_z])
            road_list.append(point)
            stair_list.append(point)
for i in range(FLOOR_GAP * (NUM_FLOORS - 1)):
    point = np.array([0, int(ROAD_WIDTH/2), i])
    road_list.append(point)
    elevator_list.append(point)
roads = np.array(road_list, dtype=np.int)
stairs = np.array(stair_list, dtype=np.int)
corridors = np.array(corridor_list, dtype=np.int)
elevators = np.array(elevator_list, dtype=np.int)

In [9]:
# goal_points = [
#             (int(TRACK_SIZE*1/4), int(ROAD_WIDTH / 2), 0),
#     (int(TRACK_SIZE*1/4), int(ROAD_WIDTH / 2), FLOOR_GAP * (NUM_FLOORS - 1))
#     ]
# goal_points = [
#             (int((TRACK_SIZE + INTERSECTION_SIZE)*1/2), int(ROAD_WIDTH / 2), 0),\
#     (int((TRACK_SIZE + INTERSECTION_SIZE)*1/2), int(ROAD_WIDTH / 2), FLOOR_GAP * (NUM_FLOORS - 1))
#     ]
# goal_points = [
#     # (0, 0, 0),\
#     #         (int(TRACK_SIZE*3/4), int(ROAD_WIDTH / 2), FLOOR_GAP * (NUM_FLOORS - 3)),\
#     #         (int(TRACK_SIZE*3/4), int(ROAD_WIDTH / 2), FLOOR_GAP * (NUM_FLOORS - 2)),\
#     (int(TRACK_SIZE*3/4), int(ROAD_WIDTH / 2), FLOOR_GAP * (NUM_FLOORS - 1)),\
#                     (int(TRACK_SIZE*1/4), int(ROAD_WIDTH / 2), FLOOR_GAP * (NUM_FLOORS - 2)),\
#                     (int(TRACK_SIZE*3/4), int(ROAD_WIDTH / 2), FLOOR_GAP * (NUM_FLOORS - 3)),\
#             (int(TRACK_SIZE*3/4), int(ROAD_WIDTH / 2), FLOOR_GAP * (NUM_FLOORS - 4)),\
#     (int(TRACK_SIZE*1/4), int(ROAD_WIDTH / 2), 0)
# ]
import random
goal_points = [
    # (0, 0, 0),\
    #         (int(TRACK_SIZE*3/4), int(ROAD_WIDTH / 2), FLOOR_GAP * (NUM_FLOORS - 3)),\
    #         (int(TRACK_SIZE*3/4), int(ROAD_WIDTH / 2), FLOOR_GAP * (NUM_FLOORS - 2)),\
    (random.randint(0,TRACK_SIZE - 1), random.randint(0,ROAD_WIDTH - 1), FLOOR_GAP * (NUM_FLOORS - 1)),\
                    (random.randint(0,TRACK_SIZE - 1), random.randint(0,ROAD_WIDTH - 1), FLOOR_GAP * (NUM_FLOORS - 2)),\
                    (random.randint(0,TRACK_SIZE - 1), random.randint(0,ROAD_WIDTH - 1), FLOOR_GAP * (NUM_FLOORS - 3)),\
            (random.randint(0,TRACK_SIZE - 1), random.randint(0,ROAD_WIDTH - 1), FLOOR_GAP * (NUM_FLOORS - 4)),\
    (random.randint(0,TRACK_SIZE - 1), random.randint(0,ROAD_WIDTH - 1), 0)
]
# goal_points = goal_points[::-1] ## Reverse the goal points

sta = time.time()
total_paths = []
total_vs = []
curr_elv_loc = CURR_ELV_LOC
for i in range(len(goal_points) - 1):
    print(i)
    s_start = goal_points[i]
    s_goal = goal_points[i+1]
    astar_instance = astar.Weighted_A_star(s_start, s_goal, roads, 1.0, NUM_FLOORS, curr_elv_loc, ELV_SPEED, ROBOT_SPEED, voxel_size[2])
    astar_instance.run()
    # print(s_start, s_goal, astar_instance.Path)
    total_paths += astar_instance.Path[::-1]
    total_vs += astar_instance.V[::-1]
    curr_elv_loc = s_goal[2]
print(time.time() - sta)

0
self.rand_num is 4
number node expanded = 1
number node expanded = 101
number node expanded = 201
1
self.rand_num is 3
number node expanded = 1
number node expanded = 101
2
self.rand_num is 3
number node expanded = 1
number node expanded = 101
3
self.rand_num is 0
number node expanded = 1
37.103888750076294


In [13]:
# Visualize the trajectory with map

x_size = int(roads[:,0].max() - roads[:,0].min()) + 1
y_size = int(roads[:,1].max() - roads[:,1].min()) + 1
z_size = int(roads[:,2].max() - roads[:,2].min()) + 1
x, y, z = np.indices((x_size, y_size, z_size))

normalized_road_xs = roads[:,0] - roads[:,0].min()
normalized_road_ys = roads[:,1] - roads[:,1].min()
normalized_road_zs = roads[:,2] - roads[:,2].min()
normalized_road = np.empty((roads.shape[0], 3), dtype=np.int)
normalized_road[:,0] = normalized_road_xs
normalized_road[:,1] = normalized_road_ys
normalized_road[:,2] = normalized_road_zs

paths = np.array(total_paths[::-1])[:,1,:]
paths = np.vstack((paths, np.array(total_paths[::-1])[-1,0]))
normalized_path_xs = paths[:,0] - roads[:,0].min()
normalized_path_ys = paths[:,1] - roads[:,1].min()
normalized_path_zs = paths[:,2] - roads[:,2].min()
normalized_path = np.empty((paths.shape[0], 3), dtype=np.int)
normalized_path[:,0] = normalized_path_xs
normalized_path[:,1] = normalized_path_ys
normalized_path[:,2] = normalized_path_zs

vertices = np.array(total_vs)
normalized_vertex_xs = vertices[:,0] - roads[:,0].min()
normalized_vertex_ys = vertices[:,1] - roads[:,1].min()
normalized_vertex_zs = vertices[:,2] - roads[:,2].min()
normalized_vertex = np.empty((vertices.shape[0], 3), dtype=np.int)
normalized_vertex[:,0] = normalized_vertex_xs
normalized_vertex[:,1] = normalized_vertex_ys
normalized_vertex[:,2] = normalized_vertex_zs

goal_points = np.array(goal_points)
normalized_gp_xs = goal_points[:,0] - roads[:,0].min()
normalized_gp_ys = goal_points[:,1] - roads[:,1].min()
normalized_gp_zs = goal_points[:,2] - roads[:,2].min()
normalized_gp = np.empty((goal_points.shape[0], 3), dtype=np.int)
normalized_gp[:,0] = normalized_gp_xs
normalized_gp[:,1] = normalized_gp_ys
normalized_gp[:,2] = normalized_gp_zs

voxels = np.zeros((x_size, y_size, z_size))
colors = np.empty(voxels.shape, dtype=object)

# for i in range(x_size):
#     for j in range(y_size):
#         for k in range(z_size):
#             if utils.check_in_2d_array(np.array([x[i,j,k], y[i,j,k], z[i,j,k]]), normalized_road):
#                 if utils.check_in_2d_array(np.array([x[i,j,k], y[i,j,k], z[i,j,k]]), normalized_gp[:1]):
#                     colors[i,j,k] = "#03C04A" # green
#                 elif utils.check_in_2d_array(np.array([x[i,j,k], y[i,j,k], z[i,j,k]]), normalized_gp[1:-1]):
#                     colors[i,j,k] = "yellow"
#                 elif utils.check_in_2d_array(np.array([x[i,j,k], y[i,j,k], z[i,j,k]]), normalized_gp[-1:]):
#                     colors[i,j,k] = "blue"
#                 # elif utils.check_in_2d_array(np.array([x[i,j,k], y[i,j,k], z[i,j,k]]), normalized_path):
#                 #     colors[i,j,k] = "red"
#                 # elif utils.check_in_2d_array(np.array([x[i,j,k], y[i,j,k], z[i,j,k]]), normalized_vertex):
#                 #     colors[i,j,k] = "#C5A3FF"
#                 else:
#                     colors[i,j,k] = "#EBECF0" # light gray
#                 voxels[i,j,k] = 1

# grad_idx = 0
# grad_start = ()
# grad_end = ()
for checked_point in roads:
    if utils.check_in_2d_array(checked_point, normalized_road):
        if utils.check_in_2d_array(checked_point, normalized_gp[:1]):
            colors[checked_point[0], checked_point[1], checked_point[2]] = (3 / 255, 192 / 255, 74 / 255) # "#03C04A" # green
        elif utils.check_in_2d_array(checked_point, normalized_gp[1:-1]):
            colors[checked_point[0], checked_point[1], checked_point[2]] = (255 / 255, 255 / 255, 20 / 255) # "yellow"
        elif utils.check_in_2d_array(checked_point, normalized_gp[-1:]):
            colors[checked_point[0], checked_point[1], checked_point[2]] = (3 / 255, 67 / 255, 223 / 255) # "blue"
        elif utils.check_in_2d_array(checked_point, normalized_path):
            colors[checked_point[0], checked_point[1], checked_point[2]] = "red"
            # grad_idx = utils.find_index_in_2d_array(checked_point, normalized_path)
            # colors[checked_point[0], checked_point[1], checked_point[2]] =\
            #  ((3 + grad_idx / (normalized_path.shape[0] - 1) * (3 - 3)) / 255 ,\
            #   (67 + grad_idx / (normalized_path.shape[0] - 1) * (192 - 67)) / 255,\
            #    (223 + grad_idx / (normalized_path.shape[0] - 1) * (74 - 223)) / 255) #"red" (229, 0, 0) # 254, 66, 15 orange
            # print(colors[checked_point[0], checked_point[1], checked_point[2]])
            # print(grad_idx)
            # grad_idx += 1
        # elif utils.check_in_2d_array(checked_point, normalized_vertex):
        #     colors[checked_point[0], checked_point[1], checked_point[2]] = (197 / 255, 163 / 255, 255 / 255) # "#C5A3FF"
        else:
            # colors[checked_point[0], checked_point[1], checked_point[2]] = (235 / 255, 236 / 255, 240 / 255) #"#EBECF0" # light gray
            colors[checked_point[0], checked_point[1], checked_point[2]] = (255/255,255/255,255/255)
        voxels[checked_point[0], checked_point[1], checked_point[2]] = 1

ax = plt.figure().add_subplot(projection='3d')
ax.grid(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
plt.axis('off')
plt.grid(b=None)
ax.voxels(voxels, facecolors=colors, edgecolor='k')

plt.show()

In [6]:
normalized_path


array([[15,  2, 20],
       [14,  2, 20],
       [13,  2, 20],
       [12,  2, 20],
       [11,  2, 20],
       [10,  2, 20],
       [ 9,  2, 20],
       [ 8,  2, 20],
       [ 7,  2, 20],
       [ 6,  2, 20],
       [ 5,  2, 20],
       [ 4,  2, 20],
       [ 3,  2, 20],
       [ 2,  2, 20],
       [ 1,  2, 20],
       [ 0,  1, 20],
       [ 0,  2, 19],
       [ 0,  2, 18],
       [ 0,  2, 17],
       [ 0,  2, 16],
       [ 0,  2, 15],
       [ 0,  2, 14],
       [ 0,  2, 13],
       [ 0,  2, 12],
       [ 0,  2, 11],
       [ 0,  1, 10],
       [ 0,  2,  9],
       [ 0,  2,  8],
       [ 0,  2,  7],
       [ 0,  2,  6],
       [ 0,  2,  5],
       [ 0,  2,  4],
       [ 0,  2,  3],
       [ 0,  2,  2],
       [ 0,  2,  1],
       [ 0,  1,  0],
       [ 1,  1,  0],
       [ 2,  1,  0],
       [ 3,  1,  0],
       [ 4,  1,  0],
       [ 5,  2, 30],
       [ 4,  2, 30],
       [ 3,  2, 30],
       [ 2,  2, 30],
       [ 1,  2, 30],
       [ 0,  1, 30],
       [ 0,  2, 29],
       [ 0,  

In [7]:
paths

array([[15.,  2., 20.],
       [14.,  2., 20.],
       [13.,  2., 20.],
       [12.,  2., 20.],
       [11.,  2., 20.],
       [10.,  2., 20.],
       [ 9.,  2., 20.],
       [ 8.,  2., 20.],
       [ 7.,  2., 20.],
       [ 6.,  2., 20.],
       [ 5.,  2., 20.],
       [ 4.,  2., 20.],
       [ 3.,  2., 20.],
       [ 2.,  2., 20.],
       [ 1.,  2., 20.],
       [ 0.,  1., 20.],
       [ 0.,  2., 19.],
       [ 0.,  2., 18.],
       [ 0.,  2., 17.],
       [ 0.,  2., 16.],
       [ 0.,  2., 15.],
       [ 0.,  2., 14.],
       [ 0.,  2., 13.],
       [ 0.,  2., 12.],
       [ 0.,  2., 11.],
       [ 0.,  1., 10.],
       [ 0.,  2.,  9.],
       [ 0.,  2.,  8.],
       [ 0.,  2.,  7.],
       [ 0.,  2.,  6.],
       [ 0.,  2.,  5.],
       [ 0.,  2.,  4.],
       [ 0.,  2.,  3.],
       [ 0.,  2.,  2.],
       [ 0.,  2.,  1.],
       [ 0.,  1.,  0.],
       [ 1.,  1.,  0.],
       [ 2.,  1.,  0.],
       [ 3.,  1.,  0.],
       [ 4.,  1.,  0.],
       [ 5.,  2., 30.],
       [ 4.,  2.

In [8]:
total_paths

[[(5.0, 2.0, 30.0), (4.0, 2.0, 30.0)],
 [(4.0, 2.0, 30.0), (3.0, 2.0, 30.0)],
 [(3.0, 2.0, 30.0), (2.0, 2.0, 30.0)],
 [(2.0, 2.0, 30.0), (1.0, 2.0, 30.0)],
 [(1.0, 2.0, 30.0), (0.0, 2.0, 30.0)],
 [(0.0, 2.0, 30.0), (0.0, 2.0, 31.0)],
 [(0.0, 2.0, 31.0), (0.0, 2.0, 32.0)],
 [(0.0, 2.0, 32.0), (0.0, 2.0, 33.0)],
 [(0.0, 2.0, 33.0), (0.0, 2.0, 34.0)],
 [(0.0, 2.0, 34.0), (0.0, 2.0, 35.0)],
 [(0.0, 2.0, 35.0), (0.0, 2.0, 36.0)],
 [(0.0, 2.0, 36.0), (0.0, 2.0, 37.0)],
 [(0.0, 2.0, 37.0), (0.0, 2.0, 38.0)],
 [(0.0, 2.0, 38.0), (0.0, 2.0, 39.0)],
 [(0.0, 2.0, 39.0), (0.0, 2.0, 40.0)],
 [(0.0, 2.0, 40.0), (1.0, 2.0, 40.0)],
 [(1.0, 2.0, 40.0), (2.0, 2.0, 40.0)],
 [(2.0, 2.0, 40.0), (3.0, 2.0, 40.0)],
 [(3.0, 2.0, 40.0), (4.0, 2.0, 40.0)],
 [(4.0, 2.0, 40.0), (5.0, 2.0, 40.0)],
 [(5.0, 2.0, 40.0), (6.0, 2.0, 40.0)],
 [(6.0, 2.0, 40.0), (7.0, 2.0, 40.0)],
 [(7.0, 2.0, 40.0), (8.0, 2.0, 40.0)],
 [(8.0, 2.0, 40.0), (9.0, 2.0, 40.0)],
 [(9.0, 2.0, 40.0), (10.0, 2.0, 40.0)],
 [(10.0, 2.0, 40.0), (11

In [5]:
# Visualize the map with segmentation

x_size = int(roads[:,0].max() - roads[:,0].min()) + 1
y_size = int(roads[:,1].max() - roads[:,1].min()) + 1
z_size = int(roads[:,2].max() - roads[:,2].min()) + 1
x, y, z = np.indices((x_size, y_size, z_size))

normalized_road_xs = roads[:,0] - roads[:,0].min()
normalized_road_ys = roads[:,1] - roads[:,1].min()
normalized_road_zs = roads[:,2] - roads[:,2].min()
normalized_road = np.empty((roads.shape[0], 3), dtype=np.int)
normalized_road[:,0] = normalized_road_xs
normalized_road[:,1] = normalized_road_ys
normalized_road[:,2] = normalized_road_zs

normalized_corridor_xs = corridors[:,0] - roads[:,0].min()
normalized_corridor_ys = corridors[:,1] - roads[:,1].min()
normalized_corridor_zs = corridors[:,2] - roads[:,2].min()
normalized_corridor = np.empty((corridors.shape[0], 3), dtype=np.int)
normalized_corridor[:,0] = normalized_corridor_xs
normalized_corridor[:,1] = normalized_corridor_ys
normalized_corridor[:,2] = normalized_corridor_zs

normalized_stair_xs = stairs[:,0] - roads[:,0].min()
normalized_stair_ys = stairs[:,1] - roads[:,1].min()
normalized_stair_zs = stairs[:,2] - roads[:,2].min()
normalized_stair = np.empty((stairs.shape[0], 3), dtype=np.int)
normalized_stair[:,0] = normalized_stair_xs
normalized_stair[:,1] = normalized_stair_ys
normalized_stair[:,2] = normalized_stair_zs

normalized_elevator_xs = elevators[:,0] - roads[:,0].min()
normalized_elevator_ys = elevators[:,1] - roads[:,1].min()
normalized_elevator_zs = elevators[:,2] - roads[:,2].min()
normalized_elevator = np.empty((elevators.shape[0], 3), dtype=np.int)
normalized_elevator[:,0] = normalized_elevator_xs
normalized_elevator[:,1] = normalized_elevator_ys
normalized_elevator[:,2] = normalized_elevator_zs

voxels = np.zeros((x_size, y_size, z_size))
colors = np.empty(voxels.shape, dtype=object)

for i in range(x_size):
    for j in range(y_size):
        for k in range(z_size):
            if utils.check_in_2d_array(np.array([x[i,j,k], y[i,j,k], z[i,j,k]]), normalized_road):
                if utils.check_in_2d_array(np.array([x[i,j,k], y[i,j,k], z[i,j,k]]), normalized_corridor):
                    colors[i,j,k] = "#ABDEE6" # green
                elif utils.check_in_2d_array(np.array([x[i,j,k], y[i,j,k], z[i,j,k]]), normalized_stair):
                    colors[i,j,k] = "#CBAACB"
                elif utils.check_in_2d_array(np.array([x[i,j,k], y[i,j,k], z[i,j,k]]), normalized_elevator):
                    colors[i,j,k] = "#FFC8A2"
                # else:
                # colors[i,j,k] = "#EBECF0" # light gray
                voxels[i,j,k] = 1

ax = plt.figure().add_subplot(projection='3d')
ax.grid(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
plt.axis('off')
plt.grid(b=None)
ax.voxels(voxels, facecolors=colors, edgecolor='k')

plt.show()